In [1]:
#import cudf
import cuml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import time
from cuml.decomposition import PCA as PCA_CUDA
from cuml import UMAP as UMAP_CUDA
from cuml import IncrementalPCA as IncPCA_CUDA
import cupy as cp
from cuml.preprocessing import MaxAbsScaler
from cuml import TSNE as TSNE_CUDA
from sklearn.preprocessing import OneHotEncoder as CPU_ohe
from sklearn.decomposition import IncrementalPCA as IncPCA

In [2]:
import gc
#import plotly.express as px

In [14]:
class GenomicVisualizer():
    
    
    def __init__(self, file_path):
        self.memory_cleared = False
        self.df_genetic_data = pd.read_hdf(path_or_buf=file_path, key="Genotypes_variants_per_sample")
        self.df_phenotype_data = pd.read_hdf(path_or_buf=file_path, key="phenotype_data")
        self.X = None
        self.performance_dict = {}
        self.components = 0
        self.columns = []
        self.X_CPU = None
        self.X_transformed_GPU = None
        self.X_transformed_CPU = None
        self.df_reduced = None
        self.label = None
        
        
    def one_hot_encode(self):
        if self.memory_cleared is False:
            if self.X is None:
                self.X = cp.asarray(CPU_ohe().fit_transform(self.df_genetic_data.values).toarray().astype(np.float32))
            else: print("The values of the genotype data frame self.df_genetic_data.values are already encoded")
        else: 
            if self.X is None:
                self.X = cp.asarray(self.X_transformed_CPU)
                self.memory_cleared = False
        
        
    def reduce_dimension(self, algorithm='PCA_CUDA', n_components=3, label=None):
        if self.memory_cleared is True: self.X = cp.asarray(self.X_CPU)
        if label not in self.df_phenotype_data.columns:
            print(f"{label} is not a phenotype. Check the columns of df_phenotype_data from the class GenomicVisualizer()")
            return
        else: self.label = label
        self.components = n_components
        if self.components not in [2, 3]:
            print("The only number of n_components / dimensions allowed are 2 or 3")
            return
        else:
            if self.components == 2: self.columns = ['component1', 'component2']
            else: self.columns = ['component1', 'component2', 'component3']
        if algorithm == 'PCA_CUDA':
            self.pca_cuda(n_components=n_components, algorithm="PCA_CUDA")
        elif algorithm == "TSNE_CUDA":
            self.tsne_cuda(n_components=n_components, algorithm="TSNE_CUDA")
        elif algorithm == "UMAP_CUDA":
            self.umap_cuda(n_components=n_components, algorithm="UMAP_CUDA")
    
    
    def pca_cuda(self, n_components, algorithm):
        start_time = time.time()
        if self.X.shape[1] <= 3000:
            self.X_transformed_GPU = PCA_CUDA(n_components=n_components).fit_transform(self.X)
        else:
            self.X_transformed_GPU = IncPCA_CUDA(n_components=n_components).fit_transform(self.X)
        self.performance_dict[algorithm] = time.time() - start_time
        self.create_reduced_dataframe()
    
    
    def tsne_cuda(self, n_components, algorithm):
        start_time = time.time()
        self.X_transformed_GPU = TSNE_CUDA(n_components=n_components, perplexity=10, n_neighbors=90).fit_transform(self.X)
        self.performance_dict[algorithm] = time.time() - start_time
        self.create_reduced_dataframe()

    
    def umap_cuda(self, n_components, algorithm):
        start_time = time.time()
        self.X_transformed_GPU = UMAP_CUDA(n_components=n_components, n_neighbors=50).fit_transform(self.X)
        self.performance_dict[algorithm] = time.time() - start_time
        self.create_reduced_dataframe()
    
    
    def create_reduced_dataframe(self):
        self.df_reduced = pd.DataFrame(self.X_transformed_GPU.get(),
                            index=self.df_genetic_data.index,
                            columns=self.columns)
        if self.label is not None:
            self.df_reduced = self.df_reduced.merge(self.df_phenotype_data[self.label], left_index=True, right_index=True)

    
    def garbage_collect(self):
        self.memory_cleared = True
        self.X_CPU = cp.asnumpy(self.X).astype(np.float32)
        self.X_transformed_CPU = cp.asnumpy(self.X_transformed_GPU).astype(np.float32)
        del self.X 
        del self.X_transformed_GPU
        gc.collect()    
        
    
    def generate_figure_image(self, save=False):
        pass


## For both the 1000 Genomes and AdaptMAP Project, the hyperparameters in TSNE_CUDA were adjusted in the class GenomicVisualizer. After each run, the reduced dataframes were stored as CSV files.

In [15]:
!pip install tables

In [16]:
genomic_visualizer = GenomicVisualizer("AdaptMap-Goat-Project.h5")
genomic_visualizer.one_hot_encode()
df_reduced_dimensions = genomic_visualizer.reduce_dimension(algorithm='UMAP_CUDA', n_components=2, label="Breeds")

In [17]:
genomic_visualizer.df_reduced.to_csv("TSNE_adaptmap_perp50.csv")
genomic_visualizer.df_reduced.head()

,component1,component2,Breeds
iid,,,
ET_ABR0001,-5.290865,6.280587,Abergelle
ET_ABR0002,-5.218983,6.214294,Abergelle
ET_ABR0003,-5.269800,6.282075,Abergelle
ET_ABR0004,-5.222088,6.291340,Abergelle
ET_ABR0005,-5.289565,6.274065,Abergelle


In [7]:
genomic_visualizer.df_reduced.to_csv("TSNE_adaptmap_perp50.csv")

In [1]:
!nvidia-smi

Thu Sep 15 22:46:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   35C    P8    19W / 230W |      0MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
genomic_visualizer.garbage_collect()
!nvidia-smi

Thu Sep 15 22:31:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   36C    P2    61W / 230W |   1405MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
genomic_visualizer = GenomicVisualizer("1000_genomes_project.h5")
genomic_visualizer.one_hot_encode()
df_reduced_dimensions = genomic_visualizer.reduce_dimension(algorithm='TSNE_CUDA', n_components=2, label="Superpopulation name")
genomic_visualizer.df_reduced.to_csv("TSNE_1000genomes_perp10.csv")
!nvidia-smi

Thu Sep 15 22:49:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   38C    P2    87W / 230W |   1399MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
genomic_visualizer.garbage_collect()
!nvidia-smi

## For both the 1000 Genomes and AdaptMAP Project, the hyperparameters in UMAP_CUDA were adjusted in the class GenomicVisualizer. After each run, the reduced dataframes were stored as CSV files

In [14]:
genomic_visualizer = GenomicVisualizer("AdaptMap-Goat-Project.h5")
genomic_visualizer.one_hot_encode()
df_reduced_dimensions = genomic_visualizer.reduce_dimension(algorithm='UMAP_CUDA', n_components=2, label="Breeds")
genomic_visualizer.df_reduced.to_csv("UMAP_adaptmap_neighbors50.csv")
!nvidia-smi

Thu Sep 15 23:01:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   40C    P2    62W / 230W |   4197MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
genomic_visualizer = GenomicVisualizer("1000_genomes_project.h5")
genomic_visualizer.one_hot_encode()
df_reduced_dimensions = genomic_visualizer.reduce_dimension(algorithm='UMAP_CUDA', n_components=2, label="Superpopulation name")
genomic_visualizer.df_reduced.to_csv("UMAP_1000genomes_neighbors5.csv")
!nvidia-smi

Thu Sep 15 23:05:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   37C    P2    30W / 230W |   1451MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------